# **Project - Eclectics Bank AI Assistant**

## **Key Features of Eclectics Bank AI Assistant**

### 💳 **Account Balance Inquiry**
- Allows users to check their account balance with:
  - Account name, balance amount, and last transaction.
- Alerts users if the account number is invalid or not found.

### 🏦 **Step-by-step Loan Application**
- Guides users through:
  1. Selecting loan type (e.g., personal, business, mortgage).
  2. Providing required details (amount, tenure, income).
  3. Submitting necessary documents.
- Ensures complete applications by validating all required inputs.

### 🧾 **Transaction History Report**
- Generates a detailed report file: `accountNumber_transactionReport.txt`.
- Report includes:
  - Transaction details (date, time, amount, type).
  - Account summary with available balance and recent activity.

### 📊 **Generate Account Summary Report**
- Summarizes all accounts into a single file: `account_summary_report.txt`.
- Includes all user account details for review or administrative purposes.

### 🔐 **Secure Login and Authentication**
- Implements secure login using:
  - Multi-Factor Authentication (MFA).
  - OTP-based verification for sensitive actions.
- Ensures user account security during interactions.

### 🛠️ **Modular Banking Services**
- Integrated tools for:
  - Balance inquiries
  - Loan applications
  - Generating reports
- Easily extensible to include new banking features.

### 💬 **Interactive Chat Interface**
- Real-time customer support via Gradio.
- Answers queries about services, transactions, and account management.
- Provides clear, professional responses with a friendly tone.

### 🛡️ **Error Handling and Validation**
- Validates user inputs for transactions and applications.
- Prevents errors like invalid account numbers or incomplete loan forms.
- Displays informative error messages to improve user experience.

### 🪑 **Personalized Recommendations**
- Suggests banking products based on:
  - Account activity
  - User preferences and financial goals.
- Helps users make informed decisions about savings and investments.


In [15]:
import os
import json
import random
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"  # or "gpt-3.5-turbo", etc.
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [16]:
###############################################################################
# 1) System Prompt
###############################################################################
system_message = (
    "You are a helpful assistant for a bank called Eclectics Bank.\n\n"
    "When the user wants to perform a banking task, follow these steps:\n\n"
    "1. **Account Balance Inquiry**\n"
    "   - If the user asks to check their account balance, request their account number.\n"
    "   - Call the function 'get_account_balance' with the provided account number.\n"
    "     - If the account number is invalid or not found, respond with: 'Invalid account number or not found. Please try again.'\n"
    "     - If valid, show the account balance, last transaction date, and the account holder's name.\n\n"
    "2. **Transaction History**\n"
    "   - If the user wants to view their transaction history, ask for their account number.\n"
    "   - Call 'get_transaction_history' with the provided account number.\n"
    "     - If no transactions are found, say: 'No transactions available for this account.'\n"
    "     - If transactions are found, list them in chronological order, showing date, type, and amount.\n\n"
    "3. **Loan Application**\n"
    "   - If the user wants to apply for a loan, guide them step by step:\n"
    "     1. Ask for the loan type (e.g., personal, business, mortgage).\n"
    "     2. Ask for the loan amount and repayment period.\n"
    "     3. Collect additional information like monthly income, employment status, and any required documents.\n"
    "   - Call 'process_loan_application' with the collected details.\n"
    "     - If the application is incomplete, notify the user and ask for the missing details.\n"
    "     - Once complete, confirm submission and provide a reference number.\n\n"
    # "4. **Generate Reports**\n"
    # "   - If the user requests a report, you can call one of the following functions:\n"
    # "     - 'generate_account_summary': Summarizes account balances and recent activity.\n"
    # "     - 'generate_transaction_report': Produces a detailed transaction history.\n"
    # "   - Provide the report in a downloadable file format (e.g., PDF or TXT).\n\n"
    "IMPORTANT:\n"
    "- Always validate account numbers and inputs before proceeding.\n"
    "- If a user requests a feature you don't recognize, say: 'I'm sorry, I don't have information about that feature.'\n"
    "- Provide clear, polite, and professional responses.\n"
    "- Maintain a short and user-friendly tone in all interactions.\n\n"
    "Remember:\n"
    "- Accuracy and security are paramount. Do not invent information.\n"
    "- For secure actions like generating reports or submitting applications, ensure the user provides all required details.\n"
    "- If you don't know something, admit it and redirect the user to bank support.\n"
)

In [3]:
###############################################################################
# 2) Banking Services Availability with Details
###############################################################################
bank_services = {
    "account_balance": [
        {
            "service": "Balance Inquiry",
            "description": "Check your account balance in real-time.",
            "processing_time": "Instant"
        }
    ],
    "transaction_history": [
        {
            "service": "Transaction History",
            "description": "View a detailed history of your transactions.",
            "processing_time": "Up to 5 minutes"
        }
    ],
    "loan_application": [
        {
            "service": "Personal Loan",
            "description": "Apply for a personal loan with flexible terms.",
            "processing_time": "24-48 hours"
        },
        {
            "service": "Business Loan",
            "description": "Get a business loan tailored to your needs.",
            "processing_time": "48-72 hours"
        },
        {
            "service": "Mortgage",
            "description": "Apply for a home mortgage with competitive rates.",
            "processing_time": "3-5 business days"
        }
    ],
    "report_generation": [
        {
            "service": "Account Summary Report",
            "description": "Generate a summary report of your account activities.",
            "processing_time": "Instant"
        },
        {
            "service": "Transaction Report",
            "description": "Get a detailed transaction report in a downloadable format.",
            "processing_time": "Instant"
        }
    ],
    "customer_support": [
        {
            "service": "24/7 Customer Support",
            "description": "Get assistance with any banking-related queries.",
            "processing_time": "Instant response"
        }
    ],
}


In [4]:
# A global list of banking service requests
bank_service_requests = []

In [5]:
###############################################################################
# 3) Helper Functions
###############################################################################
def generate_reference_numbers(seed_value):
    """
    Generate unique reference numbers for banking service requests.
    """
    random.seed(seed_value)
    return [
        f"REF-{random.randint(10000, 99999)}"
        for _ in range(5)
    ]

def get_account_balance(service_name: str):
    """
    Return the details for a given banking service from 'service_availability'.
    If the service is not available, return an empty list.
    """
    print(f"[TOOL] check_service_availability({service_name})")
    service = service_name.lower()
    return service_availability.get(service, [])

def generate_service_ticket(service_request, request_number):
    """
    Create a text file: firstName_lastName_requestNumber.txt
    containing service request details.
    """
    fname = service_request["first_name"].replace(" ", "_")
    lname = service_request["last_name"].replace(" ", "_")
    filename = f"{fname}_{lname}_{request_number}.txt"

    content = (
        "Banking Service Request\n"
        "========================\n"
        f"Request #   : {request_number}\n"
        f"Customer    : {service_request['first_name']} {service_request['last_name']}, Age {service_request['age']}\n"
        f"Service     : {service_request['service_name']}\n"
        f"Branch      : {service_request['branch']}\n"
        f"Reference # : {service_request['reference']}\n"
    )
    with open(filename, "w") as f:
        f.write(content)

    print(f"[TOOL] Service ticket generated => {filename}")
    return filename

def book_service(service_name, branch, first_name, last_name, age):
    """
    Book a banking service request.
    - Validate service availability.
    - Create new service request record, generate reference number, and save ticket file.
    """
    print(f"[TOOL] book_service({service_name=}, {branch=})")

    available_services = check_service_availability(service_name)
    if not available_services:
        return f"Error: No such service available: {service_name.title()}."

    # Generate a reference number
    reference_list = generate_reference_numbers(hash(service_name + branch + str(len(bank_service_requests))))
    chosen_reference = reference_list[0]

    new_request = {
        "service_name": service_name.title(),
        "branch":       branch.title(),
        "reference":    chosen_reference,
        "first_name":   first_name.title(),
        "last_name":    last_name.title(),
        "age":          age,
    }
    bank_service_requests.append(new_request)

    request_number  = len(bank_service_requests)
    ticket_filename = generate_service_ticket(new_request, request_number)

    confirmation = (
        f"Request #{request_number} confirmed for {first_name.title()} {last_name.title()}. "
        f"Service: {service_name.title()} at {branch.title()} branch. "
        f"Reference: {chosen_reference}. Ticket saved to {ticket_filename}."
    )
    print(f"[TOOL] {confirmation}")
    return confirmation

def generate_report():
    """
    Summarize ALL banking service requests in a single file called service_summary_report.txt.
    """
    print(f"[TOOL] generate_report called.")

    report_content = "Banking Service Request Summary Report\n"
    report_content += "=======================================\n"

    if not bank_service_requests:
        report_content += "No service requests found.\n"
    else:
        for i, request in enumerate(bank_service_requests, start=1):
            report_content += (
                f"Request #   : {i}\n"
                f"Customer    : {request['first_name']} {request['last_name']}, Age {request['age']}\n"
                f"Service     : {request['service_name']}\n"
                f"Branch      : {request['branch']}\n"
                f"Reference # : {request['reference']}\n"
                "-------------------------\n"
            )

    filename = "service_summary_report.txt"
    with open(filename, "w") as f:
        f.write(report_content)

    msg = f"Summary report generated => {filename}"
    print(f"[TOOL] {msg}")
    return msg


In [6]:
###############################################################################
# 4) Tools JSON Schemas
###############################################################################

account_balance_function = {
    "name": "check_account_balance",
    "description": (
        "Check the balance of the user's account, including the account name, "
        "balance amount, and last transaction details."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "account_number": {
                "type": "string",
                "description": "The user's account number to check balance."
            },
        },
        "required": ["account_number"],
    },
}

loan_application_function = {
    "name": "apply_for_loan",
    "description": (
        "Guide the user through a loan application process by selecting the loan type, "
        "providing necessary details, and submitting required documents."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "loan_type": {
                "type": "string",
                "description": "Type of loan (e.g., personal, business, mortgage)."
            },
            "loan_amount": {
                "type": "number",
                "description": "Amount requested for the loan."
            },
            "loan_tenure": {
                "type": "number",
                "description": "Duration of the loan (in years)."
            },
            "monthly_income": {
                "type": "number",
                "description": "User's monthly income for loan eligibility."
            },
            "documents": {
                "type": "array",
                "description": "List of documents required for loan application.",
                "items": {
                    "type": "string",
                    "description": "Document names or types."
                }
            }
        },
        "required": ["loan_type", "loan_amount", "loan_tenure", "monthly_income"],
    },
}

transaction_history_function = {
    "name": "get_transaction_history",
    "description": (
        "Generate a detailed report of all transactions for a user's account, "
        "including date, time, amount, and type of transaction."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "account_number": {
                "type": "string",
                "description": "The user's account number to fetch transaction history."
            },
        },
        "required": ["account_number"],
    },
}

generate_account_summary_function = {
    "name": "generate_account_summary_report",
    "description": (
        "Generate a summary report of all accounts, including user account details for administrative purposes."
    ),
    "parameters": {
        "type": "object",
        "properties": {
        },
        "required": [],
    },
}

secure_login_function = {
    "name": "secure_user_login",
    "description": (
        "Authenticate the user using multi-factor authentication (MFA) and OTP-based verification for secure login."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "username": {
                "type": "string",
                "description": "The user's username for login."
            },
            "password": {
                "type": "string",
                "description": "The user's password."
            },
            "otp": {
                "type": "string",
                "description": "One-time password (OTP) sent to the user for verification."
            }
        },
        "required": ["username", "password", "otp"],
    },
}

tools = [
    {"type": "function", "function": account_balance_function},
    {"type": "function", "function": loan_application_function},
    {"type": "function", "function": transaction_history_function},
    {"type": "function", "function": generate_account_summary_function},
    {"type": "function", "function": secure_login_function},
]


In [7]:
###############################################################################
# 5) Handle Tool Calls
###############################################################################

def handle_tool_call(message):
    """
    The LLM can request to call a function in 'tools'. We parse the JSON arguments
    and run the Python function. Then we return a 'tool' message with the result.
    """
    tool_call = message.tool_calls[0]
    fn_name   = tool_call.function.name
    args      = json.loads(tool_call.function.arguments)

    if fn_name == "check_account_balance":
        account_number = args.get("account_number")
        balance_info = check_account_balance(account_number)
        if not balance_info:
            response_content = {"account_number": account_number, "balance": "No account found."}
        else:
            response_content = {
                "account_number": account_number,
                "balance": balance_info["balance"],
                "last_transaction": balance_info.get("last_transaction", "N/A")
            }

    elif fn_name == "apply_for_loan":
        loan_type = args.get("loan_type")
        loan_amount = args.get("loan_amount")
        loan_tenure = args.get("loan_tenure")
        monthly_income = args.get("monthly_income")
        documents = args.get("documents", [])

        loan_status = apply_for_loan(loan_type, loan_amount, loan_tenure, monthly_income, documents)
        response_content = {
            "loan_type": loan_type,
            "loan_amount": loan_amount,
            "loan_tenure": loan_tenure,
            "monthly_income": monthly_income,
            "status": loan_status
        }

    elif fn_name == "get_transaction_history":
        account_number = args.get("account_number")
        transaction_history = get_transaction_history(account_number)
        if not transaction_history:
            response_content = {"account_number": account_number, "transactions": "No transactions found."}
        else:
            response_content = {
                "account_number": account_number,
                "transactions": transaction_history
            }

    elif fn_name == "generate_account_summary_report":
        summary_report = generate_account_summary_report()
        response_content = {"summary_report": summary_report}

    elif fn_name == "secure_user_login":
        username = args.get("username")
        password = args.get("password")
        otp = args.get("otp")

        login_status = secure_user_login(username, password, otp)
        if login_status:
            response_content = {"status": "Login successful", "username": username}
        else:
            response_content = {"status": "Login failed", "error": "Invalid credentials or OTP."}

    else:
        response_content = {"error": f"Unknown tool: {fn_name}"}

    return {
        "role": "tool",
        "content": json.dumps(response_content),
        "tool_call_id": tool_call.id,
    }, args


In [8]:
###############################################################################
# 6) Main Chat Function
###############################################################################
def chat(message, history):
    """
    The main chat loop that handles the conversation with the user,
    passing 'tools' definitions to the LLM for function calling.
    """
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    try:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )

        # If the LLM requests a function call, handle it
        while response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message
            print(f"[INFO] Tool call requested: {msg.tool_calls[0]}")
            tool_response, tool_args = handle_tool_call(msg)
            print(f"[INFO] Tool response: {tool_response}")

            # Add both the LLM's request and our tool response to the conversation
            messages.append(msg)
            messages.append(tool_response)

            # Re-send updated conversation to get final or next step
            response = openai.chat.completions.create(
                model=MODEL,
                messages=messages
            )

        # Return normal text response (finish_reason = "stop")
        return response.choices[0].message.content

    except Exception as e:
        print(f"[ERROR] {e}")
        return "I'm sorry, something went wrong while processing your request."

In [19]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [20]:
###############################################################################
# 7) Launch Gradio
###############################################################################
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': "You are a helpful assistant for a bank called Eclectics Bank.\n\nWhen the user wants to perform a banking task, follow these steps:\n\n1. **Account Balance Inquiry**\n   - If the user asks to check their account balance, request their account number.\n   - Call the function 'get_account_balance' with the provided account number.\n     - If the account number is invalid or not found, respond with: 'Invalid account number or not found. Please try again.'\n     - If valid, show the account balance, last transaction date, and the account holder's name.\n\n2. **Transaction History**\n   - If the user wants to view their transaction history, ask for their account number.\n   - Call 'get_transaction_history' with the provided account number.\n     - If no transactions are found, say: 'No transactions available for this account.'\n     - If transactions are found, list them in chronological order, showing date, type, and amount.\n